In [1]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimlab.data_profile.' + cim_profile)


In [2]:
from cimlab.loaders import Parameter, ConnectionParameters
from cimlab.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimlab.loaders.gridappsd import GridappsdConnection, get_topology_response
from cimlab.models.distributed_model import DistributedModel
# import cim.data_profile as cim
import json


['/home/d3m614/gridappsd-cim-profile', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/d3m614/.local/lib/python3.8/site-packages', '/home/d3m614/gridappsd-python', '/home/d3m614/CIMHub/src_python', '/home/d3m614/gridappsd-cim-profile', '/home/d3m614/topology-processor', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


In [3]:
# # Blazegraph connection for running inside the container
# params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
# bg = BlazegraphConnection(params)

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

In [5]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [6]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [7]:
# from gridappsd import GridAPPSD
from topology_processor import DistributedTopology

gapps = GridappsdConnection(feeder_mrid)
Topology = DistributedTopology(gapps, feeder_mrid)
topology_response = Topology.create_switch_areas(feeder_mrid)
# topology_response = get_topology_response(feeder_mrid)
topology_response = json.loads(topology_response)
# with open('topo_message.json') as topo_file:
#     topology_response = json.load(topo_file)


/home/d3m614/.local/lib/python3.8/site-packages/ipykernel_launcher.py


In [8]:
# print(topology_response)

In [9]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response['feeders'])

In [10]:
switch_area = network.switch_areas[0]

# Get phase, bus info about ACLineSegments
switch_area.get_all_attributes(cim.ACLineSegment)
switch_area.get_all_attributes(cim.ACLineSegmentPhase)
switch_area.get_all_attributes(cim.Terminal)

line_ids = list(switch_area.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', switch_area.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in switch_area.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
    

line mrid:  _0BBD0EA3-F665-465B-86FD-FC8B8466AD53
line name: 645646
bus 1:  _8C58660F-C62C-4903-BE72-22F1255B1E62
bus 2:  _0124E881-B82D-4206-BBDF-37D585159872
phase  C :  _94FCAFFF-E5F7-4F97-B697-35C46656EDCA
phase  B :  _CE2AFB05-A1B1-4082-B4EB-2AD8A1D3422D
line mrid:  _1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0
line name: 632645
bus 1:  _0124E881-B82D-4206-BBDF-37D585159872
bus 2:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  C :  _19B6044B-6915-4245-9DD1-6AB0444D42A6
phase  B :  _D9C9CEA6-B3F0-4E02-9911-3DFA192B4D01
line mrid:  _A04CDFB1-E951-4FC4-8882-0323CD70AE3C
line name: 650632
bus 1:  _673E896A-DCBF-4E43-9924-BEB31C5B6005
bus 2:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  A :  _2A5A1025-AD81-4BFA-A7F0-ED70F7F43DAA
phase  B :  _2D60B241-DE87-48BD-A8D3-FE4B705B40ED
phase  C :  _A66508A9-C9C1-4DB5-8FFD-F081E5E9E14D
line mrid:  _FBE26B35-13AB-457D-9795-DF58B28E309D
line name: 632633
bus 1:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
bus 2:  _C6256170-E6ED-4F91-8EBD-748090C3FDD5
phase  A :

In [11]:
switch_area = network.switch_areas[0]

switch_area.get_all_attributes(cim.EnergyConsumer)
switch_area.get_all_attributes(cim.EnergyConsumerPhase)

load_ids = list(switch_area.typed_catalog[cim.EnergyConsumer].keys())

for load in load_ids:
    print('p = ', switch_area.typed_catalog[cim.EnergyConsumer][load].p)
    print('q = ', switch_area.typed_catalog[cim.EnergyConsumer][load].q)
    for load_phs in switch_area.typed_catalog[cim.EnergyConsumer][load].EnergyConsumerPhase:
        print('phases: ', load_phs.phase)

p =  230000
q =  132000
phases:  B
p =  170000
q =  125000
phases:  B


In [12]:
network.get_all_attributes(cim.TransformerTank)
network.get_all_attributes(cim.TransformerTankEnd)
network.get_all_attributes(cim.TransformerTankInfo)
network.get_all_attributes(cim.TransformerEndInfo)
network.get_all_attributes(cim.ShortCircuitTest)
network.get_all_attributes(cim.NoLoadTest)

In [13]:
network.get_all_attributes(cim.ACLineSegment)
network.get_all_attributes(cim.ACLineSegmentPhase)
network.get_all_attributes(cim.PerLengthPhaseImpedance)
network.get_all_attributes(cim.PhaseImpedanceData)

network.get_all_attributes(cim.WireSpacingInfo)
network.get_all_attributes(cim.WirePosition)
network.get_all_attributes(cim.OverheadWireInfo)
network.get_all_attributes(cim.ConcentricNeutralCableInfo)
network.get_all_attributes(cim.TapeShieldCableInfo)

network.get_all_attributes(cim.Terminal) #bus info

In [14]:
for line in network.typed_catalog[cim.ACLineSegment].values():
    print()
    print('line mrid: ', line.mRID)
    print('line name:', line.name)
    print('bus 1: ', line.Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', line.Terminals[1].ConnectivityNode.mRID)
    
    for line_phs in line.ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
        if line_phs.WireInfo is not None:
            print('type: ', line_phs.WireInfo.__class__.__name__)
            print('gmr: ', line_phs.WireInfo.gmr)
            print('insulated: ', line_phs.WireInfo.insulated)
            
    if line.WireSpacingInfo is not None:
        for position in line.WireSpacingInfo.WirePositions:
            print('seq:', position.sequenceNumber, ' x:', position.xCoord, ' y:', position.yCoord)    
            
    if line.PerLengthImpedance is not None:
        for impedance in line.PerLengthImpedance.PhaseImpedanceData:
            print('row:', impedance.row, 'col:', impedance.column, 'r:', impedance.r, 'x:', impedance.x, 'b:', impedance.b)



line mrid:  _0BBD0EA3-F665-465B-86FD-FC8B8466AD53
line name: 645646
bus 1:  _8C58660F-C62C-4903-BE72-22F1255B1E62
bus 2:  _0124E881-B82D-4206-BBDF-37D585159872
phase  C :  _94FCAFFF-E5F7-4F97-B697-35C46656EDCA
phase  B :  _CE2AFB05-A1B1-4082-B4EB-2AD8A1D3422D
row: 2 col: 2 r: 0.00082607345 x: 0.00083707202 b: 0
row: 2 col: 1 r: 0.0001283788 x: 0.00028527931 b: 0
row: 1 col: 1 r: 0.00082259367 x: 0.00084316162 b: 0

line mrid:  _1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0
line name: 632645
bus 1:  _0124E881-B82D-4206-BBDF-37D585159872
bus 2:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  C :  _19B6044B-6915-4245-9DD1-6AB0444D42A6
phase  B :  _D9C9CEA6-B3F0-4E02-9911-3DFA192B4D01
row: 2 col: 2 r: 0.00082607345 x: 0.00083707202 b: 0
row: 2 col: 1 r: 0.0001283788 x: 0.00028527931 b: 0
row: 1 col: 1 r: 0.00082259367 x: 0.00084316162 b: 0

line mrid:  _45395C84-F20A-4F5A-977F-B80348256421
line name: 670671
bus 1:  _30BE5988-DE57-4E0C-AB08-50D5A13D2C1B
bus 2:  _DC889FA5-7B28-4273-A1D7-205BE3E0BFED
p

In [15]:
(cim.ConcentricNeutralCableInfo))

SyntaxError: unmatched ')' (4117457807.py, line 1)

In [16]:
for oh_wire in network.typed_catalog[cim.OverheadWireInfo].values():
    print()
    print('name: ', oh_wire.name)
    print('gmr: ', oh_wire.gmr)
    print('insulated:', oh_wire.insulated)
    for line_phs in oh_wire.ACLineSegmentPhases:
        node1 = line_phs.ACLineSegment.Terminals[0].ConnectivityNode
        node2 = line_phs.ACLineSegment.Terminals[1].ConnectivityNode
        print('Buses:', node1.name, node2.name)
        print('Line Phase: ', line_phs.name, line_phs.mRID)

KeyError: <class 'cimlab.data_profile.rc4_2021.gridappsd_cim_profile_rc4_2021.OverheadWireInfo'>

In [17]:
for cable in network.typed_catalog[cim.TapeShieldCableInfo].values():
    print()
    print('name: ', cable.name)
    print('gmr: ', cable.gmr)
    print('insulated:', cable.insulated)
    print('tape thickness', cable.tapeThickness)
    for line_phs in cable.ACLineSegmentPhases:
        node1 = line_phs.ACLineSegment.Terminals[0].ConnectivityNode
        node2 = line_phs.ACLineSegment.Terminals[1].ConnectivityNode
        print('Line Phase: ', line_phs.name, line_phs.mRID)
        print('Buses:', node1.name, node2.name, node1.mRID, node2.mRID)

KeyError: <class 'cimlab.data_profile.rc4_2021.gridappsd_cim_profile_rc4_2021.TapeShieldCableInfo'>

In [18]:
for spacing in network.typed_catalog[cim.WireSpacingInfo].values():
    for positions in spacing.WirePositions:
        print('seq:', position.sequenceNumber, ' x:', position.xCoord, ' y:', position.yCoord)    
    for line in spacing.ACLineSegments:
        node1 = line.Terminals[0].ConnectivityNode
        node2 = line.Terminals[1].ConnectivityNode
        print('Line: ', line.name, line.mRID)
        print('Buses:', node1.name, node2.name, node1.mRID, node2.mRID)

KeyError: <class 'cimlab.data_profile.rc4_2021.gridappsd_cim_profile_rc4_2021.WireSpacingInfo'>

In [19]:
for impedance in network.typed_catalog[cim.PerLengthPhaseImpedance].values():
    print('name:', impedance.name)
    for data in impedance.PhaseImpedanceData:
            print('row:', data.row, 'col:', data.column, 'r:', data.r, 'x:', data.x, 'b:', data.b)
    for line in impedance.ACLineSegments:
        node1 = line.Terminals[0].ConnectivityNode
        node2 = line.Terminals[1].ConnectivityNode
        print('Line: ', line.name, line.mRID)
        print('Buses:', node1.name, node2.name, node1.mRID, node2.mRID)

name: mtx602
row: 3 col: 1 r: 0.000096936556 x: 0.00031175045 b: 0
row: 1 col: 1 r: 0.00046765675 x: 0.000734108 b: 0
row: 3 col: 2 r: 0.000098179333 x: 0.00026322003 b: 0
row: 3 col: 3 r: 0.00046206425 x: 0.00075262537 b: 0
row: 2 col: 2 r: 0.00046448767 x: 0.00074460946 b: 0
row: 2 col: 1 r: 0.000095383086 x: 0.00023917231 b: 0
Line:  632633 _FBE26B35-13AB-457D-9795-DF58B28E309D
Buses: 632 633 _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B _C6256170-E6ED-4F91-8EBD-748090C3FDD5
name: mtx603
row: 2 col: 2 r: 0.00082607345 x: 0.00083707202 b: 0
row: 2 col: 1 r: 0.0001283788 x: 0.00028527931 b: 0
row: 1 col: 1 r: 0.00082259367 x: 0.00084316162 b: 0
Line:  645646 _0BBD0EA3-F665-465B-86FD-FC8B8466AD53
Buses: 646 645 _8C58660F-C62C-4903-BE72-22F1255B1E62 _0124E881-B82D-4206-BBDF-37D585159872
Line:  632645 _1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0
Buses: 645 632 _0124E881-B82D-4206-BBDF-37D585159872 _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
name: mtx601
row: 2 col: 1 r: 0.000095383086 x: 0.00023917231 b: 0
r

In [ ]:
json.loads(network.__dumps__(cim.PerLengthPhaseImpedance))

In [ ]:
print(network.get_attributes_query(cim.TapeShieldCableInfo))

In [ ]:
secondary_area = network.switch_areas[4].secondary_areas[0]
secondary_area.get_all_attributes(cim.PowerElectronicsConnection)
secondary_area.get_all_attributes(cim.PowerElectronicsConnectionPhase)

pec_ids = list(secondary_area.typed_catalog[cim.PowerElectronicsConnection])

for pec in pec_ids:
    print('p = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].p)
    print('q = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].q)
    for pec_phs in secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].PowerElectronicsConnectionPhases:
        print('phase ', pec_phs.phase, ': ', pec_phs.mRID)

In [ ]:
for i in a.values():
    print(i)

In [ ]:
a = {'b':'a', 'b1': 'a2'}

In [ ]:
values = ['_0649C1EF-E39F-4B46-AD80-D3C180E4562E']
message = bg.sparql.get_class_type_sparql(feeder_mrid, values)
output = bg.execute(message)
out = output['results']['bindings']
if values and not out:
    print('yes')

In [ ]:
network.switch_areas[1].typed_catalog.keys()